Install dependencies

In [ ]:
!pip install pandas
!pip install numpy
!pip install jupyter-dash
!pip install matplotlib

Import requirements

In [13]:
import pandas as pd
import plotly.graph_objects as go
import os.path

# Import chart data

Load global charts and track features

In [14]:
featureData = pd.read_csv('data/track_features.csv')
chartDataGlobal = pd.read_csv('data/chart_global.csv')

Function to import data by region group

In [15]:
def load_charts_for_codes(codes):
    temp_df = pd.DataFrame()
    for code in codes["code"]:
        if os.path.isfile("data/chart_{}.csv".format(code.lower())):
            temp_df.append(pd.read_csv("data/chart_{}.csv".format(code.lower())))
    return temp_df

Load regional chart data

In [21]:
#Europe
codes = pd.read_csv("data/EuropeCode.csv")
chartDataEurope = load_charts_for_codes(codes)

#Asia
codes = pd.read_csv('data/AsiaCode.csv')
chartDataAsia = load_charts_for_codes(codes)

#NorthAmerica
codes = pd.read_csv('data/NorthAmericaCode.csv')
chartDataNorthAmerica = load_charts_for_codes(codes)

#SouthAmerica
# codes = pd.read_csv('data/SouthAmericaCode.csv')
# chartDataSouthAmerica = load_charts_for_codes(codes)

#Africa
codes = pd.read_csv('data/AfricaCode.csv')
chartDataAfrica = load_charts_for_codes(codes)

#Ociania
codes = pd.read_csv('data/OcianiaCode.csv')
chartDataOciania = load_charts_for_codes(codes)

# Merging

## create continent charts

In [ ]:
#Europe 
# newDataGlobal = chartDataGlobal.merge(featureData, left_on='track_id', right_on='track_id')
# newDataGlobal = newDataGlobal.groupby('date').mean()

# test1 = pd.read_csv(f'/home/ole/Documents/Coding/Spotilytics/data/chart_de.csv')
# test2 = pd.read_csv(f'/home/ole/Documents/Coding/Spotilytics/data/chart_at.csv')

# test1.where()

# testData = test1.merge(test2, left_on='date', right_on='date')
# newtest = newtest.merge(featureData, left_on='track_id', right_on='track_id')
# newtest = newtest.groupby('date').mean()



#Global
newDataGlobal = chartDataGlobal.merge(featureData, left_on='track_id', right_on='track_id')
newDataGlobal = newDataGlobal.groupby('date').mean()

# europeDurationFigure = go.Figure()
# europeDurationFigure.add_trace(go.Scatter(x=newDataGlobal.index, y=newDataGlobal.danceability, mode='lines', name='global'))
# europeDurationFigure.add_trace(go.Scatter(x=testData.index, y=testData.danceability, mode='lines', name='Europe'))
# europeDurationFigure.show()


## prepare data of countries


Function to join chart tracks and track features

In [22]:
def join_chart_features(chart):
    temp_df = pd.DataFrame()
    for track in chart:
        info = track.merge(featureData, left_on='track_id', right_on='track_id')
        temp_df.append([track.country, info.groupby('date').mean()])
    return temp_df

Join features and tracks per region

In [23]:
newDataEurope = join_chart_features(chartDataEurope)
newDataAsia = join_chart_features(chartDataAsia)
newDataNorthAmerica = join_chart_features(chartDataNorthAmerica)
# newDataSouthAmerica = join_chart_features(chartDataSouthAmerica)
newDataAfrica = join_chart_features(chartDataAfrica)
newDataOciania = join_chart_features(chartDataOciania)

In [ ]:
# Average continents
# for trackInfo in chartDataEurope:
    

# Vis with Plotly

In [ ]:
#danceability	energy	key	loudness	mode	speechiness	acousticness	instrumentalness	liveness	valence	tempo	duration_ms	time_signature


## Graph Helper Fuctions

In [ ]:
def showLineFigure(dataGlobal, dataContinent, featureName, continentName):
    newLineFigure = go.Figure()
    newLineFigure.add_trace(go.Scatter(x=dataGlobal.index, y=dataGlobal.danceability, mode='lines', name='global'))
    for country in dataContinent:
        newLineFigure.add_trace(go.Scatter(x=country[1].index, y=country[1].danceability, mode='lines', name=country[0][1]))
    newLineFigure.update_layout(
        xaxis_title="Date",
        yaxis_title=featureName,
        legend_title=continentName,
    )
    newLineFigure.show()

# DO NO USE :D It destroys your CPU
def showBarFigure(dataGlobal, dataContinent, featureName, continentName):
    newBarFigure = go.Figure()
    newBarFigure.add_trace(go.Bar(x=dataGlobal.index, y=dataGlobal.danceability, name='global'))
    for country in dataContinent:
        newBarFigure.add_trace(go.Bar(x=country[1].index, y=country[1].danceability, name=country[0][1]))
    newBarFigure.update_layout(
        xaxis_title="Date",
        yaxis_title=featureName,
        legend_title=continentName,
    )
    newBarFigure.show()


## danceability

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "danceability", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "danceability", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "danceability", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "danceability", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "danceability", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "danceability", "Ociania")

## energy

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "energy", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "energy", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "energy", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "energy", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "energy", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "energy", "Ociania")

## key

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "key", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "key", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "key", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "key", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "key", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "key", "Ociania")

## loudness

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "loudness", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "loudness", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "loudness", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "loudness", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "loudness", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "loudness", "Ociania")

## mode

In [ ]:
# europeDurationFigure = go.Figure()
# europeDurationFigure.add_trace(go.Scatter(x=newDataGlobal.index, y=newDataGlobal.mode, mode='lines', name='global'))
# # for country in newDataEurope:
# #     europeDurationFigure.add_trace(go.Scatter(x=country[1].index, y=country[1].mode, mode='lines', name=country[0][1]))
# europeDurationFigure.show()

## speechiness

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "speechiness", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "speechiness", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "speechiness", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "speechiness", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "speechiness", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "speechiness", "Ociania")

## acousticness

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "acousticness", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "acousticness", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "acousticness", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "acousticness", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "acousticness", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "acousticness", "Ociania")

## instrumentalness

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "instrumentalness", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "instrumentalness", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "instrumentalness", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "instrumentalness", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "instrumentalness", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "instrumentalness", "Ociania")

## liveness

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "liveness", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "liveness", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "liveness", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "liveness", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "liveness", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "liveness", "Ociania")

## valence

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "valence", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "valence", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "valence", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "valence", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "valence", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "valence", "Ociania")

## tempo

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "tempo", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "tempo", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "tempo", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "tempo", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "tempo", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "tempo", "Ociania")

## Duration

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "Duration", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "Duration", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "Duration", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "Duration", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "Duration", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "Duration", "Ociania")

## time_signature

In [ ]:
showLineFigure(newDataGlobal, newDataEurope, "time_signature", "Europe")
showLineFigure(newDataGlobal, newDataAsia, "time_signature", "Asia")
showLineFigure(newDataGlobal, newDataNorthAmerica, "time_signature", "North America")
showLineFigure(newDataGlobal, newDataSouthAmerica, "time_signature", "South America")
showLineFigure(newDataGlobal, newDataAfrica, "time_signature", "Africa")
showLineFigure(newDataGlobal, newDataOciania, "time_signature", "Ociania")